# Turma #867 - Projeto Machine Learning II

- Equipe 1
    - Marcio Henrique dos Santos Silveira
    - Frederico Fernandes Soares
    - Bruno Baring
    - Marilia Fernandes Costa
    - Luísa Callegari de Oliveira


### Investigação

In [88]:
# Importando Bibliotecas
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.naive_bayes import MultinomialNB, GaussianNB, CategoricalNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier

# Ver todas as colunas
pd.set_option('display.max_columns', None)

# Ver todas as linhas
# pd.set_option('display.max_rows', None)

In [89]:
# Carregando o CSV
df = pd.read_csv('credito-imoveis/application_train.csv')

# Mostrar as primeiras linhas
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,456162,0,Cash loans,F,N,N,0,112500.0,700830.0,22738.5,585000.0,Unaccompanied,Working,Incomplete higher,Single / not married,House / apartment,0.019689,-8676,-813,-4163.0,-1363,NaN,1,1,1,1,0,0,Core staff,1.0,2,2,FRIDAY,17,0,0,0,1,1,0,Trade: type 2,NaN,0.699373,0.171468,0.0619,0.0302,0.9762,0.6736,0.0055,0.0,0.1034,0.1667,0.0417,0.0000,0.0504,0.0507,0.0,0.0,0.0630,0.0313,0.9762,0.6864,0.0055,0.0,0.1034,0.1667,0.0417,0.0000,0.0551,0.0528,0.0,0.0,0.0625,0.0302,0.9762,0.6780,0.0055,0.0,0.1034,0.1667,0.0417,0.0000,0.0513,0.0516,0.0,0.0,reg oper account,block of flats,0.0399,Block,No,0.0,0.0,0.0,0.0,-589.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,134978,0,Cash loans,F,N,N,0,90000.0,375322.5,14422.5,324000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.025164,-13583,-223,-3554.0,-3287,NaN,1,1,0,1,0,0,High skill tech staff,2.0,2,2,MONDAY,11,0,0,0,0,0,0,Business Entity Type 3,0.541385,0.199651,0.768808,0.0227,0.0566,0.9806,0.7348,0.0161,0.0,0.1034,0.0417,0.0833,0.0133,0.0185,0.0184,0.0,0.0,0.0231,0.0587,0.9806,0.7452,0.0162,0.0,0.1034,0.0417,0.0833,0.0136,0.0202,0.0192,0.0,0.0,0.0229,0.0566,0.9806,0.7383,0.0162,0.0,0.1034,0.0417,0.0833,0.0135,0.0188,0.0187,0.0,0.0,reg oper account,block of flats,0.0158,Block,No,0.0,0.0,0.0,0.0,-1409.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
2,318952,0,Cash loans,M,Y,N,0,180000.0,544491.0,16047.0,454500.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-13993,-6202,-7971.0,-4175,9.0,1,1,1,1,0,0,Managers,2.0,2,2,THURSDAY,15,0,0,0,0,0,0,Business Entity Type 1,NaN,0.704880,0.626304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

# Valores faltantes
## Imputação de missings    
Para garantir robustez, adicione uma camada de imputação de missing para todas as features. Reflita sobre qual é a melhor estratégia para cada uma das variáveis explicativas.

In [90]:
# Verificando dados faltantes
nulos = df.isna().sum()
mask = nulos > 0
analise_nulos = nulos[mask]
analise_nulos.sort_values(ascending=False)

COMMONAREA_MEDI             171978
COMMONAREA_AVG              171978
COMMONAREA_MODE             171978
NONLIVINGAPARTMENTS_MEDI    170914
NONLIVINGAPARTMENTS_MODE    170914
                             ...  
EXT_SOURCE_2                   544
AMT_GOODS_PRICE                226
AMT_ANNUITY                     10
DAYS_LAST_PHONE_CHANGE           1
CNT_FAM_MEMBERS                  1
Length: 67, dtype: int64

# Seleção de features
## Utilizar pelo menos uma variável categórica e pelo menos uma variável numérica.

### Escolhendo as variáveis explicativas do Modelo
* A base contém mais de 100 colunas, com o intuito de delimitar e escolher as variáveis explicativas, decidimos utilizar o modelo de árvore de decisão e o cálculo de feature importance para nos ajudar nessa escolha

In [91]:
# Definição das variáveis explicativas (categóricas e numéricas)
var_expl_cat = df.select_dtypes(include='object').drop(['ORGANIZATION_TYPE'], axis=1).columns.to_list()
var_expl_num = df.select_dtypes(include=['float64', 'int64']).drop(['TARGET','SK_ID_CURR'], axis=1).columns.to_list()
var_expl = var_expl_cat + var_expl_num

# Definição da variável resposta (Target)
var_resp = ['TARGET']

In [92]:
df.dropna(subset = var_expl, inplace=True)

# Separando os dados entre treino e teste
# Utilizamos: Base de teste = 20% dos dados
# random_state = 1 para garantir a reprodutibilidade
df_treino, df_teste = train_test_split(df, test_size = 0.2, random_state = 1)

x_treino = df_treino[var_expl].copy()
x_teste = df_teste[var_expl].copy()

y_treino = df_treino[var_resp].copy()
y_teste = df_teste[var_resp].copy()

print(x_treino.shape, x_teste.shape, y_treino.shape, y_teste.shape)

(5480, 119) (1370, 119) (5480, 1) (1370, 1)


In [93]:
# Tratando dados categóricos em numéricos, utilizando o OrdinalEncoder da biblioteca do Scikit-Learn 
# Este estimador transforma cada recurso categórico em um novo recurso de inteiros (0 a n_categorias - 1)
oe = OrdinalEncoder()
x_treino[var_expl_cat] = oe.fit_transform(x_treino[var_expl_cat])
x_teste[var_expl_cat] = oe.transform(x_teste[var_expl_cat])

In [94]:
# Instanciando e aplicando a normalização dos dados - Trazendo dados para a mesma escala/base do Z-Score
scaler = StandardScaler()
x_treino_std = scaler.fit_transform(x_treino)
x_teste_std = scaler.transform(x_teste)

In [95]:
# Instanciar o modelo
modelo1 = DecisionTreeClassifier(max_leaf_nodes = 20, random_state = 1)

# Treinar o modelo
modelo1.fit(x_treino_std, y_treino)

# Gerar as predições
y_teste_pred = modelo1.predict_proba(x_teste_std)[:, 1]

In [96]:
# Selecionar as features que irão compor o modelo de Machine Learning, através do Feature Importances
feature_imp = pd.Series({i:j for i, j in zip(x_treino.columns, modelo1.feature_importances_)})
feature_imp.sort_values(ascending=False).head(20)

EXT_SOURCE_1                  0.220070
EXT_SOURCE_2                  0.180645
EXT_SOURCE_3                  0.147728
DAYS_BIRTH                    0.108458
DAYS_LAST_PHONE_CHANGE        0.075272
REGION_POPULATION_RELATIVE    0.049240
AMT_ANNUITY                   0.043282
NONLIVINGAPARTMENTS_MEDI      0.039587
AMT_REQ_CREDIT_BUREAU_YEAR    0.038129
LIVINGAREA_MEDI               0.035101
DAYS_EMPLOYED                 0.031562
LANDAREA_AVG                  0.030927
YEARS_BUILD_MEDI              0.000000
LIVINGAPARTMENTS_MEDI         0.000000
LANDAREA_MEDI                 0.000000
AMT_REQ_CREDIT_BUREAU_WEEK    0.000000
AMT_REQ_CREDIT_BUREAU_MON     0.000000
AMT_REQ_CREDIT_BUREAU_QRT     0.000000
FLOORSMIN_MEDI                0.000000
FLOORSMAX_MEDI                0.000000
dtype: float64

Através da análise de feature importance, optamos por analisar o modelo pelas variáveis explicativas:
- DAYS_BIRTH
- DAYS_LAST_PHONE_CHANGE
- REGION_POPULATION_RELATIVE
- AMT_ANNUITY
- NONLIVINGAPARTMENTS_MEDI
- AMT_REQ_CREDIT_BUREAU_YEAR
- LIVINGAREA_MEDI
- DAYS_EMPLOYED
- LANDAREA_AVG

In [97]:
# Criando um novo dataset com as variáveis que escolhemos + target
df = df[['EXT_SOURCE_1',
         'EXT_SOURCE_2',
         'EXT_SOURCE_3',
         'DAYS_BIRTH',
         'DAYS_LAST_PHONE_CHANGE',
         'REGION_POPULATION_RELATIVE',
         'AMT_ANNUITY',
         'NONLIVINGAPARTMENTS_MEDI',
         'AMT_REQ_CREDIT_BUREAU_YEAR',
         'LIVINGAREA_MEDI',
         'DAYS_EMPLOYED',
         'LANDAREA_AVG',
         'NAME_EDUCATION_TYPE',
         'NAME_FAMILY_STATUS',
         'TARGET']]
# Mostrar as primeiras linhas
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6850 entries, 38 to 245917
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   EXT_SOURCE_1                6850 non-null   float64
 1   EXT_SOURCE_2                6850 non-null   float64
 2   EXT_SOURCE_3                6850 non-null   float64
 3   DAYS_BIRTH                  6850 non-null   int64  
 4   DAYS_LAST_PHONE_CHANGE      6850 non-null   float64
 5   REGION_POPULATION_RELATIVE  6850 non-null   float64
 6   AMT_ANNUITY                 6850 non-null   float64
 7   NONLIVINGAPARTMENTS_MEDI    6850 non-null   float64
 8   AMT_REQ_CREDIT_BUREAU_YEAR  6850 non-null   float64
 9   LIVINGAREA_MEDI             6850 non-null   float64
 10  DAYS_EMPLOYED               6850 non-null   int64  
 11  LANDAREA_AVG                6850 non-null   float64
 12  NAME_EDUCATION_TYPE         6850 non-null   object 
 13  NAME_FAMILY_STATUS          68

### Preparação para rodar o modelo de RandomForestClassifier
- Conisderando as variáveis escolhidas acima

In [98]:
# Definição das variáveis explicativas
var_expl = ['EXT_SOURCE_1',
         'EXT_SOURCE_2',
         'EXT_SOURCE_3',
         'DAYS_BIRTH',
         'DAYS_LAST_PHONE_CHANGE',
         'REGION_POPULATION_RELATIVE',
         'AMT_ANNUITY',
         'NONLIVINGAPARTMENTS_MEDI',
         'AMT_REQ_CREDIT_BUREAU_YEAR',
         'LIVINGAREA_MEDI',
         'DAYS_EMPLOYED',
         'LANDAREA_AVG',
         'NAME_EDUCATION_TYPE',
         'NAME_FAMILY_STATUS']

# Definição da variável resposta (Target)
var_resp = ['TARGET']

# Pré processamento

## Pipeline

**União do pré-processamento com o modelo**

Consequentemente tiramos as métricas aqui também.

In [99]:
# rodar otimizacao de hiperparametros simples para so depois que escolher o melhor modelo, 
# fazer um fine tunning para melhorar e chegar no roc_auc > 0.75

In [100]:
# Separando os dados entre treino e teste
# Utilizamos: Base de teste = 20% dos dados
# random_state = 1 para garantir a reprodutibilidade
df_treino, df_teste = train_test_split(df, test_size = 0.2, random_state = 1)

x_treino = df_treino[var_expl].copy()
x_teste = df_teste[var_expl].copy()

y_treino = df_treino[var_resp].copy()
y_teste = df_teste[var_resp].copy()

print(x_treino.shape, x_teste.shape, y_treino.shape, y_teste.shape)

(5480, 14) (1370, 14) (5480, 1) (1370, 1)


# Seleção de modelo pela ROC-AUC
## Lembrar de fazer otimização de hiperparâmetros

In [101]:
# Teste com vários modelos para escolher qual o que tem o melhor desempenho no menor tempo
modelos = [RandomForestClassifier(),
            DecisionTreeClassifier(),
            SVC(probability=True),
            LogisticRegression(),
            AdaBoostClassifier(),
            KNeighborsClassifier(),
            GradientBoostingClassifier(),
            HistGradientBoostingClassifier()
            ]

for modelo in modelos:
    start = time.time()

    pipe_education_marital = Pipeline(steps = [('trata_education', OrdinalEncoder())])

    pipe_selecao = ColumnTransformer(transformers = [('pipe_education', pipe_education_marital, ['NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS'])])

    pipe = Pipeline(steps = [
        ('ordinal_encoder', pipe_selecao),
        ('impute_missings', SimpleImputer(strategy='mean', add_indicator=True)),
        ('scaler', StandardScaler()),
        ('model', modelo) 
    ])
    # Treinar o modelo
    pipe.fit(x_treino, y_treino.values.ravel())

    # Gerar as predições
    y_pred_teste = pipe.predict_proba(x_teste)[:, 1]
    elapsed_time = time.time() - start
    print(f"Modelo: {modelo}; roc_auc: {roc_auc_score(y_teste, y_pred_teste)}; Tempo de execução: {elapsed_time}")

Modelo: RandomForestClassifier(); roc_auc: 0.557379947962715; Tempo de execução: 0.23601508140563965
Modelo: DecisionTreeClassifier(); roc_auc: 0.5578935845942691; Tempo de execução: 0.011960029602050781
Modelo: SVC(probability=True); roc_auc: 0.5661201909717836; Tempo de execução: 0.8461158275604248
Modelo: LogisticRegression(); roc_auc: 0.5266375325233031; Tempo de execução: 0.014364004135131836
Modelo: AdaBoostClassifier(); roc_auc: 0.5605459704785242; Tempo de execução: 0.11890983581542969
Modelo: KNeighborsClassifier(); roc_auc: 0.5200486691759079; Tempo de execução: 0.02852916717529297
Modelo: GradientBoostingClassifier(); roc_auc: 0.5587692929497056; Tempo de execução: 0.1461789608001709
Modelo: HistGradientBoostingClassifier(); roc_auc: 0.5529845656402353; Tempo de execução: 0.3792598247528076


### LogisticRegression se saiu melhor

## Treino e Teste

In [ ]:
penalty = {"l1", "l2", "elasticnet", "none"}
C = [0, 0.5, 1]

In [51]:
df_treino, df_teste = train_test_split(df, test_size = 0.15, random_state = 1)

In [76]:
x_treino

,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_BIRTH,DAYS_LAST_PHONE_CHANGE,REGION_POPULATION_RELATIVE,AMT_ANNUITY,NONLIVINGAPARTMENTS_MEDI,AMT_REQ_CREDIT_BUREAU_YEAR,LIVINGAREA_MEDI,DAYS_EMPLOYED,LANDAREA_AVG
185920,0.523029,0.183416,0.474051,-15478,-1832.0,0.020246,23755.5,0.0000,3.0,0.1251,-4264,0.0478
15941,0.672340,0.710552,0.775155,-15736,-2236.0,0.072508,48631.5,0.0039,1.0,0.0914,-618,0.0000
116227,0.525851,0.507179,0.761026,-15183,-2121.0,0.006305,47056.5,0.0000,1.0,0.0181,-1504,0.0208
82817,0.164079,0.695238,0.432962,-9483,-1514.0,0.024610,18400.5,0.0000,2.0,0.0012,-248,0.0023
9961,0.337588,0.268869,0.166406,-18028,-172.0,0.015221,10125.0,0.0000,3.0,0.0375,-998,0.0489
...,...,...,...,...,...,...,...,...,...,...,...,...
31709,0.489971,0.607254,0.401407,-13681,-692.0,0.020713,30631.5,0.0000,4.0,0.0253,-2894,0.0234
186074,0.273907,0.634435,0.320163,-11549,-536.0,0.010006,49297.5,0.0039,0.0,0.1345,-3477,0.0783
142499,0.554137,0.658463,0.427657,-13102,-1351.0,0.019689,29839.5,0.0000,0.0,0.2345,-1737,0.1666
8442,0.387798,0.752440,0.574447,-17329,0.0,0.031329,34627.5,0.0000,1.0,0.5488,-5443,0.3601


In [ ]:
%%time

parametros = {
    'modelo__penalty': ["l1", "l2", "elasticnet", "none"],
    'modelo__C': [0, 0.5, 1],
}

modelo = LogisticRegression()


lista_ordenada = [
    'Lower secondary',
    'Secondary / secondary special', 
    'Incomplete higher',
    'Higher education', 
    'Academic degree', 
]

pipe_education = Pipeline(steps = [
    ('trata_education', OrdinalEncoder(categories = [lista_ordenada]))
])

pipe_flag = Pipeline(steps = [
    ('trata_flag', OrdinalEncoder())
])

pipe_selecao = ColumnTransformer(transformers = [
    ('pipe_education', pipe_education, ['NAME_EDUCATION_TYPE']),
    ('pipe_flag', pipe_flag, ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']),
    ('outras_colunas', 'passthrough', ['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED']),
])

pipeline = Pipeline(steps = [
    ('preproc', pipe_selecao),
    ('modelo', modelo)
])

gscv = GridSearchCV(
    estimator = pipeline,
    param_grid = parametros,
    scoring = 'roc_auc',
    refit = True,
    cv = 3
)

gscv.fit(x_treino, y_treino)
y_pred_treino = gscv.predict_proba(x_treino)[:, 1]
y_pred_teste = gscv.predict_proba(x_teste)[:, 1]

In [ ]:
pipe_selecao.fit_transform(x_treino)

In [ ]:
roc_auc_score(y_treino, y_pred_treino)

In [ ]:
roc_auc_score(y_teste, y_pred_teste)

## Previsão

In [ ]:
y_pred_oculto = pipeline.predict_proba(x_oculto)[:, 1]
df_oculto['Y_PRED'] = y_pred_oculto
df_oculto[['SK_ID_CURR', 'Y_PRED']].head()